In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

## Plot
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import matplotlib as plt

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

In [3]:
df = pd.read_csv('Processed_Data.csv', engine = 'python')
df

,Unnamed: 0,topic,tweet,sentiment,class,processed_tweets
0,0,#olympics,Aussies would be happy that the T20 series hap...,0.275,Positive,aussie would happy series happen midst olympic...
1,1,#olympics,The worst thing about the #Olympics finishing ...,-0.13333333333333333,Negative,worst thing olympics finish whole week availab...
2,2,#olympics,#Olympics\n\nWe play for India: #Hockey captai...,0.0,Neutral,olympics play india hockey captain ranirampal ...
3,3,#olympics,See the best moments from the #Tokyo2020 closi...,1.0,Positive,see best moment tokyo close ceremony videoelep...
4,4,#olympics,Fabulous! #Olympics \n#LoveTheBBC \n\nTokyo Ol...,0.5,Positive,fabulous olympics lovethebbc tokyo olympics bb...
...,...,...,...,...,...,...
140253,140242,Tokyo olympics,Congratulations to all our winners and partici...,0.5,Positive,congratulation winner participant olympics win...
140254,140243,Tokyo olympics,I hope that I am wrong but I have seen no twee...,-0.5,Negative,hope wrong see tweet government
140255,140244,Tokyo olympics,Tokyo passes the baton to Paris as strangest e...,0.0,Neutral,tokyo pass baton paris strangest ever olympic ...
140256,140245,Tokyo olympics,"Paris plans to deliver inclusive, youth-centre...",0.0,Neutral,paris plan deliver inclusive youth centre gend...


In [4]:
df = df.dropna()
df

,Unnamed: 0,topic,tweet,sentiment,class,processed_tweets
0,0,#olympics,Aussies would be happy that the T20 series hap...,0.275,Positive,aussie would happy series happen midst olympic...
1,1,#olympics,The worst thing about the #Olympics finishing ...,-0.13333333333333333,Negative,worst thing olympics finish whole week availab...
2,2,#olympics,#Olympics\n\nWe play for India: #Hockey captai...,0.0,Neutral,olympics play india hockey captain ranirampal ...
3,3,#olympics,See the best moments from the #Tokyo2020 closi...,1.0,Positive,see best moment tokyo close ceremony videoelep...
4,4,#olympics,Fabulous! #Olympics \n#LoveTheBBC \n\nTokyo Ol...,0.5,Positive,fabulous olympics lovethebbc tokyo olympics bb...
...,...,...,...,...,...,...
140253,140242,Tokyo olympics,Congratulations to all our winners and partici...,0.5,Positive,congratulation winner participant olympics win...
140254,140243,Tokyo olympics,I hope that I am wrong but I have seen no twee...,-0.5,Negative,hope wrong see tweet government
140255,140244,Tokyo olympics,Tokyo passes the baton to Paris as strangest e...,0.0,Neutral,tokyo pass baton paris strangest ever olympic ...
140256,140245,Tokyo olympics,"Paris plans to deliver inclusive, youth-centre...",0.0,Neutral,paris plan deliver inclusive youth centre gend...


In [5]:
def ratio(x):
  if x=='Positive':
    return 1
  elif x=='Negative':
    return 2
  else:
    return 0;

In [6]:
df['label'] = df['class'].apply(ratio)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Unnamed: 0,topic,tweet,sentiment,class,processed_tweets,label
0,0,#olympics,Aussies would be happy that the T20 series hap...,0.275,Positive,aussie would happy series happen midst olympic...,1
1,1,#olympics,The worst thing about the #Olympics finishing ...,-0.13333333333333333,Negative,worst thing olympics finish whole week availab...,2
2,2,#olympics,#Olympics\n\nWe play for India: #Hockey captai...,0.0,Neutral,olympics play india hockey captain ranirampal ...,0
3,3,#olympics,See the best moments from the #Tokyo2020 closi...,1.0,Positive,see best moment tokyo close ceremony videoelep...,1
4,4,#olympics,Fabulous! #Olympics \n#LoveTheBBC \n\nTokyo Ol...,0.5,Positive,fabulous olympics lovethebbc tokyo olympics bb...,1


In [7]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [8]:
df['processed_tweets'] = df['processed_tweets'].astype(str)
x = df['processed_tweets']
y = df['label']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(x)

sequences = tokenizer.texts_to_sequences(x)
data = pad_sequences(sequences, maxlen=50)

In [10]:
data.shape

(139707, 50)

In [11]:
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import confusion_matrix, accuracy_score

In [12]:

x_train,x_test,y_train,y_test = tts(data,y,test_size = 0.2)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(111765, 50)
(27942, 50)
(111765,)
(27942,)


In [13]:
def create_conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(vocabulary_size, 100, input_length=50))
    model_conv.add(Dropout(0.2))
    model_conv.add(Conv1D(64, 5, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=4))
    model_conv.add(LSTM(100))
    model_conv.add(Dense(3, activation='softmax'))
    model_conv.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_conv

In [14]:
model_conv = create_conv_model()

In [15]:
model_conv.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 100)           2000000   
_________________________________________________________________
dropout (Dropout)            (None, 50, 100)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 46, 64)            32064     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 11, 64)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               66000     
_________________________________________________________________
dense (Dense)                (None, 3)                 303       
Total params: 2,098,367
Trainable params: 2,098,367
Non-trainable params: 0
______________________________________________

In [17]:
model_conv.fit(x_train, y_train, validation_split=0.2, epochs = 10)

Epoch 1/10
2795/2795 [==============================] - 127s 45ms/step - loss: 0.4948 - accuracy: 0.8193 - val_loss: 0.4206 - val_accuracy: 0.8538
Epoch 2/10
2795/2795 [==============================] - 125s 45ms/step - loss: 0.3486 - accuracy: 0.8794 - val_loss: 0.4097 - val_accuracy: 0.8618
Epoch 3/10
2795/2795 [==============================] - 125s 45ms/step - loss: 0.2657 - accuracy: 0.9079 - val_loss: 0.4286 - val_accuracy: 0.8592
Epoch 4/10
2795/2795 [==============================] - 125s 45ms/step - loss: 0.2053 - accuracy: 0.9289 - val_loss: 0.4886 - val_accuracy: 0.8530
Epoch 5/10
2795/2795 [==============================] - 125s 45ms/step - loss: 0.1670 - accuracy: 0.9423 - val_loss: 0.5380 - val_accuracy: 0.8549
Epoch 6/10
2795/2795 [==============================] - 125s 45ms/step - loss: 0.1399 - accuracy: 0.9519 - val_loss: 0.5786 - val_accuracy: 0.8555
Epoch 7/10
2795/2795 [==============================] - 125s 45ms/step - loss: 0.1216 - accuracy: 0.9577 - val_loss: 0

In [18]:
model_conv.evaluate(x_test, y_test)

874/874 [==============================] - 6s 7ms/step - loss: 0.7033 - accuracy: 0.8435


[0.7032945156097412, 0.8435330390930176]